In [1]:
!pip install scikit-learn


In [2]:
# Upgrade pip and setuptools
!pip install --upgrade pip setuptools

# Install packages with specific versions
!pip install transformers torch scikit-learn openpyxl

import pandas as pd
from transformers import RobertaTokenizer, RobertaForSequenceClassification
from sklearn.model_selection import train_test_split
import torch
from torch.utils.data import DataLoader, TensorDataset, RandomSampler, SequentialSampler
import re
import numpy as np


  Using cached pip-24.2-py3-none-any.whl.metadata (3.6 kB)
  Using cached setuptools-72.1.0-py3-none-any.whl.metadata (6.6 kB)
Using cached pip-24.2-py3-none-any.whl (1.8 MB)
Using cached setuptools-72.1.0-py3-none-any.whl (2.3 MB)
  Attempting uninstall: setuptools
    Found existing installation: setuptools 71.0.4
    Uninstalling setuptools-71.0.4:
      Successfully uninstalled setuptools-71.0.4
  Attempting uninstall: pip
    Found existing installation: pip 24.1.2
    Uninstalling pip-24.1.2:
      Successfully uninstalled pip-24.1.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
ipython 7.34.0 requires jedi>=0.16, which is not installed.


  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl.metadata (1.8 kB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-manylinu

In [3]:
# Excel dosyasını yükleyin
file_path = '/content/sample_data/duygular_stemmed.xlsx'  # Excel dosyanızın yolu
data = pd.read_excel(file_path)

# Veri seti sütun adlarını kontrol edin
print(data.head())

# Varsayılan olarak 'text' ve 'label' sütunları olduğunu varsayıyorum; gerekirse bunları değiştirin
data.columns = ['comments', 'label']  # Sütun adlarını veri setinize göre ayarlayın

# NaN değerleri kontrol edin ve metni 'string' türüne dönüştürün
data['comments'] = data['comments'].fillna('').astype(str)

# Etiketleri sayısal değerlere dönüştürme (önceki kodun etiketleri 'string' olabilir)
data['label'] = pd.Categorical(data['label']).codes  # Etiketleri sayısal formata dönüştürün

# Metinleri küçük harfe dönüştürme
data['comments'] = data['comments'].str.lower()

# Gereksiz karakterleri kaldırma
data['comments'] = data['comments'].apply(lambda x: re.sub(r'[^a-zA-Zçğıöşü\s]', '', x))

# Boşlukları düzeltme
data['comments'] = data['comments'].apply(lambda x: re.sub(r'\s+', ' ', x).strip())

# Metin ve etiketleri ayırma
texts = data['comments'].tolist()
labels = data['label'].tolist()

# RoBERTa tokenizer'ını yükleyin
tokenizer = RobertaTokenizer.from_pretrained('roberta-base')

# Tokenizasyon, özel tokenların eklenmesi ve attention maskelerinin oluşturulması
def preprocess_data(texts, tokenizer, max_length=128):
    inputs = tokenizer(
        texts,
        max_length=max_length,
        padding='max_length',
        truncation=True,
        return_tensors='pt',
        return_attention_mask=True,
    )
    return inputs['input_ids'], inputs['attention_mask']

# Veriyi ön işleyin
input_ids, attention_masks = preprocess_data(texts, tokenizer)

# Etiketleri tensora dönüştürün
labels = torch.tensor(labels, dtype=torch.long)

# Veriyi eğitim ve test setlerine ayırın
train_inputs, test_inputs, train_labels, test_labels = train_test_split(
    input_ids, labels, test_size=0.2, random_state=42
)
train_masks, test_masks, _, _ = train_test_split(
    attention_masks, attention_masks, test_size=0.2, random_state=42
)

# Veriyi DataLoader'a dönüştürün
batch_size = 16

train_data = TensorDataset(train_inputs, train_masks, train_labels)
train_sampler = RandomSampler(train_data)
train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=batch_size)

test_data = TensorDataset(test_inputs, test_masks, test_labels)
test_sampler = SequentialSampler(test_data)
test_dataloader = DataLoader(test_data, sampler=test_sampler, batch_size=batch_size)

# RoBERTa modelini yükleyin
model = RobertaForSequenceClassification.from_pretrained('roberta-base', num_labels=6)

# Eğitim parametreleri
from transformers import AdamW

optimizer = AdamW(model.parameters(), lr=2e-5)

# Eğitim döngüsü
epochs = 3
for epoch in range(epochs):
    model.train()
    for step, batch in enumerate(train_dataloader):
        b_input_ids, b_input_mask, b_labels = batch
        optimizer.zero_grad()
        outputs = model(b_input_ids, attention_mask=b_input_mask, labels=b_labels)
        loss = outputs.loss
        loss.backward()
        optimizer.step()

    print(f"Epoch {epoch+1}/{epochs} - Loss: {loss.item()}")

# Modeli değerlendirme
model.eval()
predictions, true_labels = [], []
for batch in test_dataloader:
    b_input_ids, b_input_mask, b_labels = batch
    with torch.no_grad():
        outputs = model(b_input_ids, attention_mask=b_input_mask)
    logits = outputs.logits
    predictions.append(logits.argmax(dim=-1).cpu().numpy())
    true_labels.append(b_labels.cpu().numpy())

# Performans değerlendirme
from sklearn.metrics import classification_report, confusion_matrix, f1_score

predictions = [item for sublist in predictions for item in sublist]
true_labels = [item for sublist in true_labels for item in sublist]

print(classification_report(true_labels, predictions))
print(confusion_matrix(true_labels, predictions))

F1_Score = f1_score(true_labels, predictions, average='weighted')
print('Test Verisi Üzerindeki Model Doğruluğu:', round(F1_Score, 2))


                                            comments     label
0  gerçek siyaset böyl yapılma sinirlenme üslup s...  mutluluk
1  ömr bin yıl ols bıkma duraklama izler barış be...  mutluluk
2      çıkan konuk ara keyif sıkılma izledik böl olt  mutluluk
3  bütün mam seri begendig hat bilgisayar indirdi...  mutluluk
4  bur çıkamıyor defa izliyor hal doya iç samim s...  mutluluk


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/481 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/499M [00:00<?, ?B/s]

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:591: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch 1/3 - Loss: 0.5983883142471313
Epoch 2/3 - Loss: 1.3794288635253906
Epoch 3/3 - Loss: 0.9002400040626526
              precision    recall  f1-score   support

           0       0.94      0.89      0.91       626
           1       0.62      0.62      0.62       588
           2       0.70      0.80      0.74       599
           3       0.84      0.67      0.75       601
           4       0.76      0.68      0.72       615
           5       0.79      0.95      0.86       614

    accuracy                           0.77      3643
   macro avg       0.77      0.77      0.77      3643
weighted avg       0.78      0.77      0.77      3643

[[555  18   7  11  15  20]
 [  9 366  87  31  45  50]
 [  5  53 479   9  31  22]
 [  7  69  51 405  41  28]
 [ 12  67  61  22 421  32]
 [  5  18   4   4   2 581]]
Test Verisi Üzerindeki Model Doğruluğu: 0.77
